# INSY662 - Group Project - Predicting Airline Fares

### Problem Definition and Motivation



### Dataset Description 


### Initial Processing Steps

In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import re

# Load dataset in folder Data 
df = pd.read_csv('data/processed_data.csv')
df.head()

# keep only 20% of rows 
data = df.sample(frac=0.2, random_state=42)




In [17]:
# 1) Check data types 
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 821388 entries, 1042959 to 3223388
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   legId                              821388 non-null  object 
 1   searchDate                         821388 non-null  object 
 2   flightDate                         821388 non-null  object 
 3   startingAirport                    821388 non-null  object 
 4   destinationAirport                 821388 non-null  object 
 5   fareBasisCode                      821388 non-null  object 
 6   travelDuration                     821388 non-null  object 
 7   elapsedDays                        821388 non-null  int64  
 8   isBasicEconomy                     821388 non-null  bool   
 9   isRefundable                       821388 non-null  bool   
 10  isNonStop                          821388 non-null  bool   
 11  baseFare                           82

In [18]:
# Chekck missing values
data.isnull().sum()

legId                                    0
searchDate                               0
flightDate                               0
startingAirport                          0
destinationAirport                       0
fareBasisCode                            0
travelDuration                           0
elapsedDays                              0
isBasicEconomy                           0
isRefundable                             0
isNonStop                                0
baseFare                                 0
totalFare                                0
seatsRemaining                           0
totalTravelDistance                  60885
segmentsDepartureTimeEpochSeconds        0
segmentsDepartureTimeRaw                 0
segmentsArrivalTimeEpochSeconds          0
segmentsArrivalTimeRaw                   0
segmentsArrivalAirportCode               0
segmentsDepartureAirportCode             0
segmentsAirlineName                      0
segmentsAirlineCode                      0
segmentsEqu

In [19]:
# Missing value Ration
missing_ratio = data.isnull().sum() / len(data) * 100
missing_ratio = missing_ratio[missing_ratio > 0].sort_values(ascending=False)
missing_ratio

totalTravelDistance             7.412453
segmentsEquipmentDescription    1.889850
segmentsDistance                1.034712
dtype: float64

In [20]:
# ---------- Find dupplicates ----------------
dupplicates = data[data.duplicated()]
ratio = len(dupplicates) / len(data) * 100
print(f"There are {len(dupplicates)} out of {len(data)} rows, representing {ratio:.2f}%")

# Ratio is low, drop
data = data.drop_duplicates()
dupplicates = data[data.duplicated()]
ratio = len(dupplicates) / len(data) * 100
print(f"There are {len(dupplicates)} out of {len(data)} rows, representing {ratio:.2f}%")


There are 3885 out of 821388 rows, representing 0.47%
There are 0 out of 817503 rows, representing 0.00%


#### Data Cleaning

In [22]:
# ------Drop unecessary segment columns---------
# Reasons: redundent information or not relevant to objective of analysis
unecessary_col = ['segmentsDepartureTimeEpochSeconds', 'segmentsArrivalTimeEpochSeconds', 'segmentsAirlineCode', 'segmentsEquipmentDescription']
data = data.drop(columns = unecessary_col)


In [23]:
# ------------- Clean/Split Segment Columns ---------------

# List segment columns
segments = [col for col in data.columns if "segment" in col.lower()]
segments

# Split columns per segment and drop original column
for col in segments:
    split_cols = data[col].str.split("||", expand=True, regex=False)
    split_cols = split_cols.dropna(axis = 1, how = "all")
    split_cols.columns = [f"{col}_{i+1}" for i in range(split_cols.shape[1])]
    data = pd.concat([data, split_cols], axis = 1)
    data = data.drop(columns = [col])

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 817503 entries, 1042959 to 3223388
Data columns (total 55 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   legId                           817503 non-null  object 
 1   searchDate                      817503 non-null  object 
 2   flightDate                      817503 non-null  object 
 3   startingAirport                 817503 non-null  object 
 4   destinationAirport              817503 non-null  object 
 5   fareBasisCode                   817503 non-null  object 
 6   travelDuration                  817503 non-null  object 
 7   elapsedDays                     817503 non-null  int64  
 8   isBasicEconomy                  817503 non-null  bool   
 9   isRefundable                    817503 non-null  bool   
 10  isNonStop                       817503 non-null  bool   
 11  baseFare                        817503 non-null  float64
 12  totalFare     

In [30]:
# ------------ Correct Data Types of Segments ----------------

# List segment columns
segmentsTime_cols = [col for col in data.columns if "timeraw" in col.lower()]
for col in segmentsTime_cols:
    data[col] = pd.to_datetime(data[col], errors='coerce', utc=True)
for c in segmentsTime_cols:
    data[c] = data[c].dt.tz_localize(None)

segmentsNumerical_cols = [col for col in data.columns if "seconds" in col.lower() or "distance" in col.lower()]
for col in segmentsNumerical_cols:
    data[col] = pd.to_numeric(data[col], errors='coerce').astype('Int64')


In [34]:
# ---------Clean Other DataTypes----------------

# Transfrom data columns into correct data types
data['searchDate'] = pd.to_datetime(data['searchDate'])
data['flightDate'] = pd.to_datetime(data['flightDate'])

# Create new column for number of days between search date and flight date
data['date_diff'] = (data['flightDate'] - data['searchDate']).dt.days


# travelDuration to numeric
data['travelDuration'] = pd.to_numeric(data['travelDuration'], errors='coerce').astype('Int64')



In [35]:
#------------- Clean Travel Duration column --------------
def parse_travel_duration(s):
    if pd.isnull(s):
        return None
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?', s)
    if not match:
        return None
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    return hours + minutes / 60

data['travelDuration_hours'] = data['travelDuration'].apply(parse_travel_duration)



In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 817503 entries, 1042959 to 3223388
Data columns (total 57 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   legId                           817503 non-null  object        
 1   searchDate                      817503 non-null  datetime64[ns]
 2   flightDate                      817503 non-null  datetime64[ns]
 3   startingAirport                 817503 non-null  object        
 4   destinationAirport              817503 non-null  object        
 5   fareBasisCode                   817503 non-null  object        
 6   travelDuration                  0 non-null       Int64         
 7   elapsedDays                     817503 non-null  int64         
 8   isBasicEconomy                  817503 non-null  bool          
 9   isRefundable                    817503 non-null  bool          
 10  isNonStop                       817503 non-null  bool 

### Exploratory Data Analysis (EDA) results

### Feature engineering steps

##### Scaling
##### Encoding
##### Binning
##### Transformations
##### Handling Outliers
##### Creation of Derived Variables

In [1]:
#